# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [49]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KAPUCO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KAPUCO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KAPUCO\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# load data from database
engine = create_engine('sqlite:///MessageDatabase.db')
df = pd.read_sql_table('Messages', engine)
X = df['message']#,'genre'
Y = df.drop(columns=['id','message','original','genre'])

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
for i in range(36):
    print(classification_report(y_test.as_matrix()[:,i], y_pred[:,i]))

              precision    recall  f1-score   support

           0       0.63      0.35      0.45      1521
           1       0.82      0.94      0.87      4994
           2       0.50      0.05      0.09        39

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.65      0.45      0.47      6554
weighted avg       0.77      0.80      0.77      6554

              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5448
           1       0.83      0.37      0.51      1106

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.86      0.68      0.72      6554
weighted avg       0.87      0.88      0.86      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

C:\Users\KAPUCO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6513
           1       0.00      0.00      0.00        41

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6466
           1       0.00      0.00      0.00        88

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

              preci

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
         ])
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 500,1000,5000),
        'tfidf__use_idf': (True, False)
    }
    cv  = GridSearchCV(pipeline, param_grid=parameters,refit=True)

    return cv


In [20]:
model = build_model()

In [21]:
model.fit(X_train,y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 500, 1000, 5000), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
model.best_params_

{'tfidf__use_idf': True,
 'vect__max_df': 0.75,
 'vect__max_features': 5000,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = model.predict(X_test)

In [17]:
for i in range(36):
    print(classification_report(y_test.as_matrix()[:,i], y_pred[:,i]))

              precision    recall  f1-score   support

           0       0.63      0.41      0.49      1521
           1       0.83      0.92      0.88      4994
           2       0.32      0.31      0.32        39

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.59      0.55      0.56      6554
weighted avg       0.78      0.80      0.78      6554

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5448
           1       0.84      0.43      0.57      1106

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.87      0.71      0.75      6554
weighted avg       0.89      0.89      0.88      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

C:\Users\KAPUCO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5713
           1       0.52      0.06      0.11       841

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.70      0.53      0.52      6554
weighted avg       0.83      0.87      0.83      6554

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6110
           1       0.33      0.01      0.02       444

   micro avg       0.93      0.93      0.93      6554
   macro avg       0.63      0.50      0.49      6554
weighted avg       0.89      0.93      0.90      6554

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6259
           1       0.75      0.12      0.21       295

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.86      0.56      0.59      6554
weighted avg       0.95      0.96      0.94      6554

              preci

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [69]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize,max_df=0.75, max_features= 5000, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
for i in range(36):
    print(classification_report(y_test.as_matrix()[:,i], y_pred[:,i]))


              precision    recall  f1-score   support

           0       0.56      0.14      0.23      1521
           1       0.78      0.96      0.86      4994
           2       0.50      0.21      0.29        39

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.61      0.44      0.46      6554
weighted avg       0.73      0.77      0.71      6554

              precision    recall  f1-score   support

           0       0.91      0.96      0.94      5448
           1       0.75      0.55      0.63      1106

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.83      0.76      0.79      6554
weighted avg       0.89      0.89      0.89      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

In [67]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize,max_df=0.75, max_features= 5000, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultiOutputClassifier(MultinomialNB()))
])

pipeline.fit(X_train, y_train)
y_pred_2 = pipeline.predict(X_test)
for i in range(36):
    print(classification_report(y_test.as_matrix()[:,i], y_pred_2[:,i]))

C:\Users\KAPUCO\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
C:\Users\KAPUCO\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.66      0.43      0.52      1521
           1       0.84      0.93      0.88      4994
           2       0.00      0.00      0.00        39

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.50      0.46      0.47      6554
weighted avg       0.79      0.81      0.80      6554

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      5448
           1       0.69      0.65      0.67      1106

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.81      0.80      0.80      6554
weighted avg       0.89      0.89      0.89      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

In [76]:
performance_difference=[]
for i in range(36):
    performance_difference.append(classification_report(y_test.as_matrix()[:,i], y_pred[:,i],output_dict =True)['weighted avg']['precision'] - classification_report(y_test.as_matrix()[:,i], y_pred_2[:,i],output_dict =True)['weighted avg']['precision'])

In [81]:
sum(performance_difference)/len(performance_difference)

0.007352839487089246

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.